In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.profiler import profile, ProfilerActivity
import torch.cuda as cuda
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from torch.utils.data import TensorDataset, DataLoader

# Verificar GPU
if torch.cuda.is_available():
    print(f"GPU: {cuda.get_device_name(0)}")
    device = "cuda"
else:
    print("No GPU available, using CPU")
    device = "cpu"


digits = load_digits()
X, Y = digits.data, digits.target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=11)

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                              torch.tensor(Y_train))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                             torch.tensor(Y_test))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)


model = nn.Sequential(nn.Linear(64, 128),
                      nn.ReLU(),
                      nn.Linear(128, 10)).to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()


it = iter(train_loader)
batch = next(it)
x = batch[0].to(device)      # [64, 64]
targets = batch[1].to(device) # [64]


activities = [ProfilerActivity.CPU, ProfilerActivity.CUDA]
with profile(activities=activities,
             with_stack=True,
             record_shapes=True,
             with_modules=True) as prof:

    optimizer.zero_grad()
    outputs = model(x)
    loss = loss_fn(outputs, targets)
    loss.backward()
    optimizer.step()

    if device == "cuda":
        torch.cuda.synchronize()


trace_path = "trace_linear_windows.json"
prof.export_chrome_trace(trace_path)
print(f"\n✅ TRACE GUARDADO: {os.path.abspath(trace_path)}")
print(f"📁 Tamaño archivo: {os.path.getsize(trace_path) / 1024:.1f} KB")


print("2. Load → Selecciona el archivo trace_linear_windows.json")

